# SQL 的五十道練習：初學者友善的資料庫入門

> 資料定義語言與資料操作語言

In [1]:
%LOAD sqlite3 db=../databases/imdb.db timeout=2 shared_cache=true

## 複習一下

在第一章「簡介」我們試圖用一句話解釋 SQL：

> SQL 是 Structured Query Language 的縮寫，是一個專門針對關聯式資料庫中所儲存的資料進行查詢、定義、操作與控制的語言。

SQL 在 1970 年代由國際商業機器公司（IBM）創造，剛開發出來時候僅只是為了更有效率地「查詢」儲存於關聯式資料庫中的資料，但是到了現代，除了查詢以外像是資料的建立、更新與刪除，也都能靠著 SQL 來完成。具體來說，SQL 是由保留字（Keyword）、符號、常數與函數所組合而成的一種語言，按照使用目的可以再細分為資料查詢語言（Data Query Language, DQL）、資料定義語言（Data Definition Language, DDL）、資料操作語言（Data Manipulation Language, DML）、資料控制語言（Data Control Language, DCL）以及交易控制語言（Transaction Control Language, TCL）。

|SQL 的分類|範例|
|:---------|:----|
|資料查詢語言（Data Query Language, DQL）|`SELECT ...`|
|資料定義語言（Data Definition Language, DDL）|`CREATE ...`|
|資料操作語言（Data Manipulation Language, DML）|`UPDATE ...`|
|資料控制語言（Data Control Language, DCL）|`GRANT ...`|
|交易控制語言（Transaction Control Language, TCL）|`COMMIT`|

SQL 是一個能夠與關聯式資料庫互動的專用語言，常見的互動有四個：包含創造（Create）、查詢（Read）、更新（Update）與刪除（Delete），這四個動作又在業界與社群被簡稱為 CRUD，其中查詢對應了本書第三章「簡介」到第十一章「垂直與水平合併資料」所介紹的資料查詢語言，本章會用點到為止的篇幅介紹與「創造」、「刪除」對應的資料定義語言以及與「更新」對應的資料操作語言部分。

## 資料定義語言

資料定義語言（Data Definition Language, DDL）最主要的保留字是 `CREATE` 與 `DROP`，這意味著我們可以在資料庫創造與刪除物件。那麼有哪些物件可以被創造與刪除呢？它們分別是檢視表（Views）與資料表（Tables）。

檢視表（Views）相較於第十章「子查詢」所介紹的子查詢，差別在於子查詢的結果是用過即捨棄的，若希望重新取得子查詢結果，必須將該段 SQL 敘述以純文字檔案（可用 `.sql` 為副檔名儲存）保存起來；檢視表則可以直接將 SQL 敘述儲存在資料庫中，並且給予一個檢視表命名，想要檢視該段 SQL 敘述的查詢結果時，只需要將檢視表命名放在 `FROM` 保留字之後即可。因此，對於純粹只需要向資料庫寫作資料查詢語言的初級資料分析師來說，檢視表可以視為等同於資料表的存在，但實際上檢視表儲存的內容並不是列（Rows）與欄（Columns）所組成的二維表格，而是一段 SQL 敘述，只有在對檢視表寫作資料查詢語言時，才會執行被儲存的 SQL 敘述。簡單來說，檢視表是一種介於「子查詢」與「建立資料表」之間的功能，就像是資料表版本的「衍生計算欄位」，由於多數「非資料庫管理員」的資料分析師在公司中沒有建立資料表的權限，因此若是有建立檢視表的權限將可以滿足我們對資料表的創造需求。

```sql
SELECT *
  FROM view;
```

### 建立與刪除檢視表

使用 `CREATE VIEW` 建立檢視表並給予檢視表命名，然後再加入希望檢視表被查詢時所執行的 SQL 敘述。

```sql
CREATE VIEW view
    AS
   SQL Statement;
```

舉例來說，我們想知道不同年份 `release_year` 上映的電影平均評等有哪些年份是大於等於 8.5 的？在第九章「分組與聚合結果篩選」我們提過針對分組聚合的結果應用 `WHERE` 是不被允許的，應該要改使用分組聚合版本的 `HAVING` 保留字加上帶有聚合函數的條件。

In [2]:
SELECT release_year,
       AVG(rating) AS avg_rating
  FROM movies
 GROUP BY release_year
HAVING AVG(rating) >= 8.5;

release_year,avg_rating
1936,8.5
1972,9.2
1974,8.6
1977,8.6
1994,8.8
1999,8.54
2002,8.5
2008,8.5


在第十章「子查詢」我們提過除了前述改使用分組聚合版本的 `HAVING` 保留字加上帶有聚合函數的條件以外，也能透過子查詢來完成。

In [3]:
SELECT *
  FROM (
           SELECT release_year,
                  AVG(rating) AS avg_rating
             FROM movies
            GROUP BY release_year
       )
       AS avg_rating_by_release_year
 WHERE avg_rating >= 8.5;

release_year,avg_rating
1936,8.5
1972,9.2
1974,8.6
1977,8.6
1994,8.8
1999,8.54
2002,8.5
2008,8.5


前述透過子查詢所建立出來，像資料表一般的 `avg_rating_by_release_year` 也能創造為檢視表，與子查詢的差異在於，建立為檢視表之後是可以確實在 SQLiteStudio 的資料庫清單中看到，而非像子查詢使用後即被捨棄。

In [4]:
CREATE VIEW avg_rating_by_release_year
    AS
SELECT release_year,
       AVG(rating) AS avg_rating
  FROM movies
 GROUP BY release_year;

![](../images/view-01.png)

回到問題「我們想知道不同年份 `release_year` 上映的電影平均評等有哪些年份是大於等於 8.5 的？」就可以寫出以下的 SQL 敘述得到這個問題的答案。

In [5]:
SELECT *
  FROM avg_rating_by_release_year
 WHERE avg_rating >= 8.5;

release_year,avg_rating
1936,8.5
1972,9.2
1974,8.6
1977,8.6
1994,8.8
1999,8.54
2002,8.5
2008,8.5


使用 `DROP VIEW` 刪除檢視表，指定欲刪除的檢視表命名即可。

```sql
DROP VIEW view;
```

In [6]:
DROP VIEW avg_rating_by_release_year;

成功刪除檢視表之後，在 SQLiteStudio 的資料庫清單中就能夠看到檢視表已經不存在。

![](../images/view-02.png)

刪除檢視表以後，就無法在 `FROM` 保留字之後指定檢視表查詢。

```sql
SELECT *
  FROM avg_rating_by_release_year
 WHERE avg_rating >= 8.5;
```

```
Error while executing SQL query on database 'imdb': no such table: avg_rating_by_release_year
```

### 建立與刪除資料表

資料表是以列（Rows）與欄（Columns）所組成的二維表格，有時列也有其他別名，像是紀錄（Records）、觀測值（Observations）、元組（Tuples）等；欄的其他別名則有欄位（Fields）、變數（Variables）、屬性（Attributes）等。

![](../images/Relational_database_terms.png)

來源：<https://en.wikipedia.org/wiki/Relational_database>

不同於檢視表，資料表確實將資料儲存於二維表格中，我們有兩種方式可以建立資料表：基於 SQL 敘述從既有資料查詢衍生為資料表或者從零至一，從零至一建立資料表除了會用到資料定義語言，也會使用資料操作語言，因此可以在後面的小節一起示範，我們先採用基於 SQL 敘述從既有資料查詢衍生為資料表。使用 `CREATE TABLE` 建立資料表並給予資料表命名，然後再加入能夠生成表中資料的 SQL 敘述。

```sql
CREATE TABLE table
    AS
   SQL Statement;
```

舉例來說，前面小節所建立的檢視表，也能夠以資料表的形式創造。這裡要提醒讀者，多數「非資料庫管理員」的資料分析師在公司中沒有建立資料表的權限，在自己電腦的 SQLiteStudio 學習環境中操作才能確保順利進行。

In [7]:
CREATE TABLE avg_rating_by_release_year
    AS
SELECT release_year,
       AVG(rating) AS avg_rating
  FROM movies
 GROUP BY release_year;

建立為資料表的 `avg_rating_by_release_year` 同樣可以確實在 SQLiteStudio 的資料庫清單中看到。

![](../images/create-table-01.png)

在回答問題「我們想知道不同年份 release_year 上映的電影平均評等有哪些年份是大於等於 8.5 的？」就可以如同建立完檢視表後寫出以下的 SQL 敘述得到這個問題的答案。

In [8]:
SELECT *
  FROM avg_rating_by_release_year
 WHERE avg_rating >= 8.5;

release_year,avg_rating
1936,8.5
1972,9.2
1974,8.6
1977,8.6
1994,8.8
1999,8.54
2002,8.5
2008,8.5


使用 `DROP TABLE` 刪除資料表，指定欲刪除的資料表命名即可。

```sql
DROP TABLE table;
```

In [9]:
DROP TABLE avg_rating_by_release_year;

成功刪除資料表之後，在 SQLiteStudio 的資料庫清單中就能夠看到資料表已經不存在。

![](../images/create-table-02.png)

刪除資料表以後，就無法在 `FROM` 保留字之後指定資料表查詢。

```sql
SELECT *
  FROM avg_rating_by_release_year
 WHERE avg_rating >= 8.5;
```

```
Error while executing SQL query on database 'imdb': no such table: avg_rating_by_release_year
```

## 資料操作語言

資料操作語言（Data Manipulation Language, DML）可以針對資料表進行更新。至於檢視表因為儲存的並不是資料，而是一段 SQL 敘述，想要更新檢視表，維持相同命名但替換儲存其中的 SQL 敘述，作法很單純，就是先 `DROP VIEW` 刪除檢視表、然後再重新 `CREATE VIEW` 建立相同命名的檢視表即可。

### 更新資料表

我們在前面的小節提到有兩種建立資料表的方式：基於 SQL 敘述從既有資料查詢衍生為資料表或者從零至一，我們已經示範過基於 SQL 敘述從既有資料查詢衍生為資料表的方式，現在我們可以利用資料定義語言以及資料操作語言從零至一建立資料表。首先使用 `CREATE TABLE` 建立資料表並給予資料表命名、指定欄位名稱與資料類別。

```sql
CREATE TABLE table (
    columns typeof_columns,
    ...
);
```

舉例來說，在 `imdb` 資料庫建立資料表 `favorite_movies`，資料表的三欄分別為 `title`、`release_year` 與 `rating`，資料類別依序為 `text`、`integer` 與 `real`。

In [10]:
CREATE TABLE favorite_movies (
    title text,
    release_year integer,
    rating real
);

建立好的資料表 `favorite_movies` 可以確實在 SQLiteStudio 的資料庫清單中看到。

![](../images/create-table-03.png)

這時候對新建立好的資料表 `favorite_movies` 查詢，會看到 0 筆觀測值。

In [11]:
SELECT *
  FROM favorite_movies;

Empty set (0.00 sec)

因為是從零至一建立資料表，除了 `CREATE TABLE` 以外，我們還需要使用 `INSERT INTO` 插入觀測值。

```sql
INSERT INTO table
VALUES 
       (observations);
```

其中觀測值以元組（Tuples）形式指派、以逗號分隔值，舉例來說，在資料表 `favorite_movies` 插入五筆觀測值。

In [12]:
INSERT INTO favorite_movies
VALUES 
       ('The Shawshank Redemption', 1995, 9.2),
       ('The Godfather', 1972, 9.1),
       ('The Dark Knight', 2008, 9.0),
       ('Forrest Gump', 1994, 8.8),
       ('Fight Club', 1999, 8.8);

完成插入觀測值後，對資料表 `favorite_movies` 查詢，會看到 `(5, 3)` 的外型。

In [13]:
SELECT *
  FROM favorite_movies;

title,release_year,rating
The Shawshank Redemption,1995,9.2
The Godfather,1972,9.1
The Dark Knight,2008,9
Forrest Gump,1994,8.8
Fight Club,1999,8.8


假如不想一筆一筆地輸入觀測值，資料也可以從副檔名為 `.csv` 的逗號分隔值檔案（Comma separated values）匯入，使用 SQLiteStudio 的資料匯入（Import）功能，記得將 First line respresents CSV column names 勾選方格取消勾選。

```
The Shawshank Redemption,1995,9.2
The Godfather,1972,9.1
The Dark Knight,2008,9
Forrest Gump,1994,8.8
Fight Club,1999,8.8
```

![](../images/import-01.png)

![](../images/import-02.png)

![](../images/import-03.png)

如果我們已經插入了觀測值才發現資料有誤，使用 `UPDATE table SET ...` 更新。

```sql
UPDATE table
   SET columns = values
 WHERE conditions;
```

舉例來說，The Shawshank Redemption 於 1994-10-14 在美國上映，於 1995-03-10 在台灣上映，中文片名為「刺激1995」，上映年份誤植為 1995。

In [14]:
UPDATE favorite_movies
   SET release_year = 1994
 WHERE title = 'The Shawshank Redemption';

完成更新觀測值後，對資料表 `favorite_movies` 查詢，會看到 The Shawshank Redemption 上映年份確實更新成了 1994。

In [15]:
SELECT *
  FROM favorite_movies;

title,release_year,rating
The Shawshank Redemption,1994,9.2
The Godfather,1972,9.1
The Dark Knight,2008,9
Forrest Gump,1994,8.8
Fight Club,1999,8.8


如果我們已經插入了觀測值才發現是不需要的資料，使用 `DELETE FROM table ...` 刪除指定觀測值。

```sql
DELETE FROM table
 WHERE conditions;
```

舉例來說，The Godfather 於 1972-03-24 在美國上映，對於讀者已經是太過於久遠的電影。

In [16]:
DELETE FROM favorite_movies
 WHERE title = 'The Godfather';

完成刪除觀測值後，對資料表 `favorite_movies` 查詢，會看到 The Godfather 已經確實被刪除。

In [17]:
SELECT *
  FROM favorite_movies;

title,release_year,rating
The Shawshank Redemption,1994,9.2
The Dark Knight,2008,9
Forrest Gump,1994,8.8
Fight Club,1999,8.8


如果希望刪除資料表中「所有」的觀測值，同樣使用 `DELETE FROM table`，但是不指定條件。

In [18]:
DELETE FROM favorite_movies;

完成刪除觀測值後，對資料表 `favorite_movies` 查詢，會看到 0 筆觀測值。

In [19]:
SELECT *
  FROM favorite_movies;

Empty set (0.01 sec)

In [20]:
DROP TABLE favorite_movies;

刪除資料表之後，在 SQLiteStudio 的資料庫清單中就能夠看到資料表已經不存在。

![](../images/create-table-02.png)

## 重點統整

- SQL 是一個能夠與關聯式資料庫互動的專用語言，常見的互動有四個：包含創造（Create）、查詢（Read）、更新（Update）與刪除（Delete）；與「創造」、「刪除」對應的是資料定義語言，與「更新」對應的是資料操作語言。
- 這個章節學起來的 SQL 保留字：
    - `CREATE VIEW`
    - `DROP VIEW`
    - `CREATE TABLE`
    - `DROP TABLE`
    - `INSERT INTO`
    - `UPDATE`
    - `SET`
    - `DELETE`
- 將截至目前所學的資料查詢語言（DQL）集中在一個敘述中，寫作順序必須遵從標準 SQL 的規定。

```sql
SELECT DISTINCT columns AS alias,
       CASE WHEN condition_1 THEN result_1
            WHEN condition_2 THEN result_2
            ...
            ELSE result_n END AS alias
  FROM table
 WHERE conditions
 GROUP BY columns
HAVING conditions
 ORDER BY columns DESC
 LIMIT m;
```

- 建立檢視表

```sql
CREATE VIEW view
    AS
   SQL Statement;
```

- 刪除檢視表 `DROP VIEW view;`

- 基於 SQL 敘述從既有資料查詢衍生為資料表

```sql
CREATE TABLE table
    AS
   SQL Statement;
```

- 從零至一建立資料表

```sql
CREATE TABLE table (
    columns typeof_columns,
    ...
);
```

- 刪除資料表 `DROP TABLE table;`

- 插入觀測值

```sql
INSERT INTO table
VALUES 
       (observations);
```

- 更新觀測值

```sql
UPDATE table
   SET columns = values
 WHERE conditions;
```

- 刪除指定觀測值

```sql
DELETE FROM table
 WHERE conditions;
```

- 刪除所有觀測值

```sql
DELETE FROM table;
```

## 練習題 40-45

練習題會涵蓋四個學習資料庫，記得要依據題目的需求，調整編輯器選單的學習資料庫，在自己電腦的 SQLiteStudio 寫出跟預期輸出相同的 SQL 敘述，寫作過程如果卡關了，可以參考附錄二「練習題參考解答」。

### 40. 從 `covid19` 資料庫建立一個檢視表名為 `total_confirmed_by_country_region` 記錄截至 2022-05-31 全球各國的確診人數，參考下列的預期輸出。

註：本題不需考慮 `daily_report` 內的 `Last_Update` 時間戳記，`daily_report` 的數據有效期間就是 2022-05-31。

預期輸出：(199, 2) 的檢視表 `total_confirmed_by_country_region`。

```sql
SELECT *
  FROM total_confirmed_by_country_region;
```

```
Country_Region	total_confirmed
Afghanistan	180347
Albania	276101
Algeria	265884
Andorra	42894
Angola	99761
...
```

### 41. 從 `twElection2020` 資料庫建立一個檢視表名為 `presidential_total_votes` 記錄三組候選人的總得票數，參考下列的預期輸出。

預期輸出：(3, 3) 的檢視表 `presidential_total_votes`。

```sql
SELECT * 
  FROM presidential_total_votes;
```

```
number	candidate	total_votes
1	宋楚瑜/余湘	608590
2	韓國瑜/張善政	5522119
3	蔡英文/賴清德	8170231
```

### 42. 從 `nba` 資料庫建立一個檢視表名為 `ppg_leader_by_teams`  紀錄各個球隊的得分王（生涯場均得分 `ppg` 全隊最高）是誰，參考下列的預期輸出。

預期輸出：(30, 4) 的檢視表 `ppg_leader_by_teams`。

```sql
SELECT *
  FROM ppg_leader_by_teams;
```

```
team	firstName	lastName	MAX(career_summaries.ppg)
Atlanta Hawks	Trae	Young	25.3
Boston Celtics	Jayson	Tatum	20.9
Brooklyn Nets	Kevin	Durant	27.2
Charlotte Hornets	LaMelo	Ball	18.3
Chicago Bulls	DeMar	DeRozan	20.8
Cleveland Cavaliers	Collin	Sexton	20
Dallas Mavericks	Luka	Doncic	26.4
Denver Nuggets	Nikola	Jokic	19.7
Detroit Pistons	Cade	Cunningham	17.4
Golden State Warriors	Stephen	Curry	24.3
Houston Rockets	John	Wall	19.1
Indiana Pacers	Buddy	Hield	15.9
LA Clippers	Paul	George	20.4
Los Angeles Lakers	LeBron	James	27.1
Memphis Grizzlies	Ja	Morant	21.2
Miami Heat	Jimmy	Butler	17.7
Milwaukee Bucks	Giannis	Antetokounmpo	21.8
Minnesota Timberwolves	Karl-Anthony	Towns	23.2
New Orleans Pelicans	Zion	Williamson	25.7
New York Knicks	Kemba	Walker	19.5
Oklahoma City Thunder	Shai	Gilgeous-Alexander	18.2
Orlando Magic	Franz	Wagner	15.2
Philadelphia 76ers	Joel	Embiid	26
Phoenix Suns	Devin	Booker	23.5
Portland Trail Blazers	Damian	Lillard	24.6
Sacramento Kings	De'Aaron	Fox	19.1
San Antonio Spurs	Keldon	Johnson	14.4
Toronto Raptors	Pascal	Siakam	15.7
Utah Jazz	Donovan	Mitchell	23.9
Washington Wizards	Bradley	Beal	22.1
```

### 43. 在 `nba` 資料庫新增一個資料表名為 `favorite_players`，具有三個欄位 `name`、`years_pro`、`ppg`，資料類型分別是文字（`TEXT`）、整數（`INTEGER`）與浮點數（`REAL`），參考下列的預期輸出。

預期輸出：(0, 3) 的資料表 `favorite_players`。

```sql
SELECT *
  FROM favorite_players;
```

```
name	years_pro	ppg
```

### 44. 承接上題，在 `nba` 資料庫的 `favorite_players` 資料表中新增五筆觀測值，參考下列的預期輸出。

預期輸出：(5, 3) 的資料表 `favorite_players`。

```sql
SELECT * 
  FROM favorite_players;
```

```
name	years_pro	ppg
Steve Nash	19	14.3
Michael Jordan	14	30.1
Paul Pierce	19	19.7
Kevin Garnett	21	17.8
Hakeem Olajuwon	18	21.8
```

### 45. 承接上題，在 `nba` 資料庫的 `favorite_players` 資料表將第五位球員 Hakeem Olajuwon 替換成 Tim Duncan，參考下列的預期輸出。

預期輸出：(5, 3) 的資料表 `favorite_players`。

```sql
SELECT *
  FROM favorite_players;
```

```
name	years_pro	ppg
Steve Nash	19	14.3
Michael Jordan	14	30.1
Paul Pierce	19	19.7
Kevin Garnett	21	17.8
Tim Duncan	19	19
```